In [1]:
import os
os.system('pip3 install seaborn')

0

In [2]:
## Importing required packages
import matplotlib
matplotlib.use('Agg')

### Importing torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

## Importing python packages
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### To test whether GPU instance is present in the system of not.
cuda = torch.cuda.is_available()
print('Using PyTorch version:', torch.__version__, 'CUDA:', cuda)
### If cuda is a gpu instance. If it's false then we run the program on CPU
### If cuda is a gpu instance. If it's true then we run the program on GPU
torch.manual_seed(42)
if cuda:
    torch.cuda.manual_seed(42)

Using PyTorch version: 0.3.1 CUDA: False


Now, we'll load the MNIST data. First time we may have to download the data, which can take a while.

In [3]:
### Initializing batch size
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

## Loading the train set file
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

## Loading the test set file
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

The train and test data are provided via data loaders that provide iterators over the datasets. The first element of training data (X_train) is a 4th-order tensor of size (batch_size, 1, 28, 28), i.e. it consists of a batch of images of size 1x28x28 pixels. y_train is a vector containing the correct classes ("0", "1", ..., "9") for each training digit.

In [4]:
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


#### Plotting the  first 10 training digits

In [5]:
pltsize=1
plt.figure(figsize=(10*pltsize, pltsize))

for i in range(10):
    plt.subplot(1,10,i+1)
    plt.axis('off')
    plt.imshow(X_train[i,:,:,:].numpy().reshape(28,28), cmap="gray")
    plt.title('Class: '+str(y_train[i]))

#### MLP network definition
Let's define the network as a Python class. We have to write the __init__() and forward() methods, and PyTorch will automatically generate a backward() method for computing the gradients for the backward pass.

Finally, we define an optimizer to update the model parameters based on the computed gradients. We select stochastic gradient descent (with momentum) as the optimization algorithm, and set learning rate to 0.01. Note that there are several different options for the optimizer in PyTorch that we could use instead of SGD.

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 50)
        self.fc1_drop = nn.Dropout(0.2)
        self.fc2 = nn.Linear(50, 50)
        self.fc2_drop = nn.Dropout(0.2)
        self.fc3 = nn.Linear(50, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc1_drop(x)
        x = F.relu(self.fc2(x))
        x = self.fc2_drop(x)
        return F.log_softmax(self.fc3(x))

model = Net()
if cuda:
    model.cuda()
    
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

print(model)

Net(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc1_drop): Dropout(p=0.2)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (fc2_drop): Dropout(p=0.2)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
)


#### Learning
Let's now define functions to train() and test() the model.

In [7]:
def train(epoch, log_interval=100):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [8]:
def test(loss_vector, accuracy_vector):
    model.eval()
    test_loss, correct = 0, 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(test_loader)
    loss_vector.append(test_loss)

    accuracy = 100. * correct / len(test_loader.dataset)
    accuracy_vector.append(accuracy)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))

Now we are ready to train our model using the train() function. An epoch means one pass through the whole training data. After each epoch, we evaluate the model using test().

In [9]:
os.system('date')

0

In [10]:
epochs = 10

lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(epoch)
    test(lossv, accv)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.286133
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.574557
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.053961
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.762598
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.567075
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.636146
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.785646
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.576619
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.422358
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.431734
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.282997
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.700439
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.328396
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.456864
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.174557
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.380142
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.547680
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.199731
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.555221

Test set: Average loss: 0.2322, Accuracy: 9305/10000 (93%)

Train Epoch: 9 [12800/60000 (21%)]	Loss: 0.088631
Train Epoch: 9 [16000/60000 (27%)]	Loss: 0.049552
Train Epoch: 9 [19200/60000 (32%)]	Loss: 0.104120
Train Epoch: 9 [22400/60000 (37%)]	Loss: 0.556603
Train Epoch: 9 [25600/60000 (43%)]	Loss: 0.271876
Train Epoch: 9 [28800/60000 (48%)]	Loss: 0.443873
Train Epoch: 9 [32000/60000 (53%)]	Loss: 0.047532
Train Epoch: 9 [35200/60000 (59%)]	Loss: 0.083300
Train Epoch: 9 [38400/60000 (64%)]	Loss: 0.114984
Train Epoch: 9 [41600/60000 (69%)]	Loss: 0.241781
Train Epoch: 9 [44800/60000 (75%)]	Loss: 0.083469
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.403411
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.125964
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.170172
Train Epoch: 9 [57600/60000 (96%)]	Loss: 0.233029

Test set: Average loss: 0.1076, Accuracy: 9660/10000 (97%)

Train Epoch: 10 [0/60000 (0%)]	Loss: 0.080552
Train Epoch: 10 [3200/60000 (5%)]	Loss: 0.269887
Train Epoch: 10 [6400/60000 (11%)]	Loss: 0.213379
Train Epoch: 10 [9600/60000 (16%)]	Loss: 0.3

In [ ]:
os.system('date')